In [2]:
!pip install pyscf 
import pyscf
from pyscf import gto, scf

# Define the molecule geometry
mol = gto.M(atom='Be 0 0 0\nH 0 0 1.6\nH 0 0 -1.6', basis='sto-3g')

# Compute the Hartree-Fock energy and wavefunction
mf = scf.RHF(mol)
mf.kernel()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 36.4 MB/s eta 0:00:00
converged SCF energy = -15.504085516671


-15.504085516670967

In [8]:
import pyscf
from pyscf import gto, scf

# define the molecule geometry
mol = gto.M(atom='Be 0 0 0; H 0 0 1.2; H 0 0 -1.2', basis='sto-3g')

# perform Hartree-Fock calculation
mf = scf.RHF(mol)
mf.kernel()


converged SCF energy = -15.5535864757067


-15.55358647570668

In [14]:
!pip install qiskit-nature

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
from qiskit_nature.drivers import PySCFDriver, UnitsType, Molecule
from qiskit_nature.transformers import FreezeCoreTransformer, ActiveSpaceTransformer
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter
from qiskit_nature.mappers.second_quantization import ParityMapper
from qiskit_nature.circuit.library import UCCSD
from qiskit import Aer
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import SLSQP

# Define the molecule geometry
molecule = Molecule(geometry=[['Be', [0., 0., 0.]], ['H', [0., 0., 1.6]], ['H', [0., 0., -1.6]]],
                     charge=0, multiplicity=1)

# Define the PySCF driver and run it to get the molecule data
driver = PySCFDriver(molecule=molecule, unit=UnitsType.ANGSTROM, basis='sto-3g')
problem = ElectronicStructureProblem(driver, [FreezeCoreTransformer(freeze_core=True),
                                               ActiveSpaceTransformer(num_electrons=2, num_molecular_orbitals=2)])

# Generate the qubit operator and the initial state
converter = QubitConverter(mapper=ParityMapper())
qubit_op = converter.convert(problem.second_q_ops())
init_state = problem.get_initial_state()

# Construct the UCCSD ansatz
ansatz = UCCSD(qubit_converter=converter, num_particles=problem.num_particles,
               num_spin_orbitals=problem.num_orbitals, initial_state=init_state)

# Run the VQE algorithm to find the ground state
backend = Aer.get_backend('statevector_simulator')
optimizer = SLSQP(maxiter=1000)
vqe = VQE(ansatz=ansatz, optimizer=optimizer, quantum_instance=backend)
result = vqe.compute_minimum_eigenvalue(qubit_op)
print(result)


ImportError: ignored

In [19]:
from qiskit import Aer
from qiskit.algorithms import VQE
from qiskit.circuit.library import TwoLocal
from qiskit.opflow import I, Z, X, Y
from qiskit_nature import QiskitNatureError
from qiskit_nature.algorithms import GroundStateEigensolver
from qiskit_nature.drivers import PySCFDriver
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.transformers import FreezeCoreTransformer, ActiveSpaceTransformer
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter
from qiskit_nature.mappers.second_quantization import ParityMapper
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.circuit.library import HartreeFock, UCCSD

# Define the molecule
molecule = "Be . H 2"
charge = 0
multiplicity = 1

# Define the PySCF driver
driver = PySCFDriver(molecule)

# Freeze the core orbitals
freeze_core = True
if freeze_core:
    transformer = FreezeCoreTransformer(remove_orbitals=[0, 1])
else:
    transformer = None

# Choose the active space orbitals
active_space = True
if active_space:
    orbitals = {"occupied": [2, 3], "active": [4, 5], "unoccupied": []}
    transformer = ActiveSpaceTransformer(**orbitals)

# Define the electronic structure problem
try:
    elec_structure_problem = ElectronicStructureProblem(driver, transformer=transformer)
except QiskitNatureError as ex:
    print(str(ex))
    exit()

# Get the fermionic Hamiltonian
fermionic_hamiltonian = elec_structure_problem.second_q_ops()[0]

# Define the qubit converter
mapper = ParityMapper()
converter = QubitConverter(mapper=mapper, two_qubit_reduction=True, z2symmetry_reduction=None)

# Convert the fermionic Hamiltonian to a qubit Hamiltonian
qubit_op = converter.convert(fermionic_hamiltonian)

# Define the initial state
init_state = HartreeFock(num_qubits=6, num_spin_orbitals=12, converter=converter)

# Define the ansatz
ansatz = UCCSD(num_qubits=6, num_orbitals=6, num_particles=(2, 0), converter=converter)

# Define the classical optimizer
optimizer = SLSQP(maxiter=1000)

# Define the quantum instance
backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend)

# Define the VQE algorithm
algorithm = VQE(ansatz=ansatz, optimizer=optimizer, quantum_instance=quantum_instance)

# Compute the ground state
calc = GroundStateEigensolver(converter, algorithm)
result = calc.solve(elec_structure_problem)

print(result)


<ipython-input-19-90d961ac0900>:6: NatureDeprecationWarning: The qiskit_nature.algorithms.excited_state_solvers package is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release. Instead use the qiskit_nature.second_q.algorithms.excited_state_solvers package.
  from qiskit_nature.algorithms import GroundStateEigensolver
<ipython-input-19-90d961ac0900>:6: NatureDeprecationWarning: The qiskit_nature.algorithms.pes_sampler package is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release . Please refer to 'https://github.com/Qiskit/qiskit-nature/issues/750' for additional information regarding 'extrapolators'.
  from qiskit_nature.algorithms import GroundStateEigensolver


ImportError: ignored

In [20]:
from qiskit.ignis.mitigation.measurement import (complete_meas_cal, CompleteMeasFitter)

# Generate the calibration circuits
qr = qiskit.QuantumRegister(6)
meas_calibs, state_labels = complete_meas_cal(qr=qr, circlabel='mcal')

# Execute the calibration circuits
calibration_qobj = qiskit.compile(meas_calibs, backend=backend, shots=8192)
calibration_job = backend.run(calibration_qobj)
calibration_result = calibration_job.result()

# Extract the calibration matrix
meas_fitter = CompleteMeasFitter(calibration_result, state_labels)
meas_filter = meas_fitter.filter

# Apply the reference state error mitigation
mitigated_counts = meas_filter.apply(result_counts)


ModuleNotFoundError: ignored

In [ ]:
# Apply reference state error mitigation
calibration_qobj = qiskit.compile(meas_calibs, backend=backend, shots=8192)
calibration_job = backend.run(calibration_qobj)
calibration_result = calibration_job.result()
meas_fitter = CompleteMeasFitter(calibration_result, state_labels)
meas_filter = meas_fitter.filter
mitigated_counts = meas_filter.apply(result_counts)


In [ ]:
import qiskit
from qiskit import Aer
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.chemistry.core import Hamiltonian, TransformationType, QubitMappingType
from qiskit.chemistry.drivers import PySCFDriver
from qiskit.chemistry.algorithms.ground_state_solvers import GroundStateEigensolver
from qiskit.chemistry.algorithms.pes_samplers import BOPESSampler
from qiskit.circuit.library import TwoLocal
from qiskit.aqua.algorithms import NumPyMinimumEigensolver
from qiskit.aqua.operators import Z2Symmetries, WeightedPauliOperator
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter

# Define molecule and driver
molecule = 'Be . H2'
charge = 0
spin = 0
driver = PySCFDriver(atom=molecule, charge=charge, spin=spin, basis='sto3g')

# Define qubit reduction and mapping
transformer = driver.run()
qmolecule = transformer.get_transformation(TransformationType.FULL, QubitMappingType.PARITY, 
                                            two_qubit_reduction=True, freeze_core=True)
num_particles = qmolecule.num_alpha + qmolecule.num_beta
num_spin_orbitals = qmolecule.num_orbitals * 2

# Create the Hamiltonian and print basic info
hamiltonian = Hamiltonian(qubit_mapping=QubitMappingType.PARITY,
                          two_qubit_reduction=True, freeze_core=True,
                          orbital_reduction=[-3.0, -1.0])
hamiltonian.from_qmol(qmolecule)
print(hamiltonian.print_details())

# Define initial state
init_state = HartreeFock(num_spin_orbitals, num_particles, 'parity')

# Define ansatz
ansatz = TwoLocal(num_spin_orbitals, ['ry', 'rz'], 'cz', reps=3, 
                  entanglement='linear', insert_barriers=True)

# Define backend
backend = Aer.get_backend('statevector_simulator')

# Define classical eigensolver
classical_solver = NumPyMinimumEigensolver()

# Define quantum eigensolver
quantum_solver = GroundStateEigensolver(hamiltonian, ansatz, NumPyMinimumEigensolver(),
                                        quantum_instance=backend)

# Define BOPESSampler
sampler = BOPESSampler(hamiltonian, initial_point=[0.01]*ansatz.num_parameters,
                       n_samples=1000, n_init_samples=100, min_prob_clip=0.001)

# Compute reference energies
reference_energy = classical_solver.compute_minimum_eigenvalue(hamiltonian).eigenvalue.real
print(f"Reference energy: {reference_energy:.6f}")

# Compute and apply calibration matrix for error mitigation
calibration_circuits, state_labels = CompleteMeasFitter(backend, qubits=ansatz.qubits).get_calibration()
job = backend.run(calibration_circuits, shots=10000)
results = job.result()
calibration_matrix = CompleteMeasFitter(backend, qubits=ansatz.qubits, state_labels=state_labels).fit(results).cal_matrix

# Define VQE object with reference-state error mitigation
vqe = qiskit.algorithms.VQE(ansatz, quantum_solver, sampler,
                            initial_point=[0.01]*ansatz.num_parameters,
                            callback=print_optimization_callback,
                            mitigation_matrix=calibration_matrix)

# Run VQE


In [ ]:
# Set up the VQE algorithm
vqe_algorithm = VQE(
    ansatz=ansatz,
    optimizer=optimizer,
    quantum_instance=quantum_instance,
    initial_point=initial_params,
    callback=callback,
)

# Run the VQE algorithm
result = vqe_algorithm.compute_minimum_eigenvalue(operator=hamiltonian)


In [21]:
# Get the ground state energy
ground_state_energy = result.eigenvalue.real

# Get the optimized parameters of the ansatz
opt_params = result.optimal_parameters

# Get the ground state wavefunction
wavefunction = ansatz.assign_parameters(opt_params)


NameError: ignored

In [ ]:
# Calculate the bond length
distance = calculate_distance(molecule.geometry[1], molecule.geometry[2])
bond_length = distance / BOHR_TO_ANGSTROM
print("Bond length: {:.2f} Å".format(bond_length))

# Calculate the dipole moment
dipole_moment = calculate_dipole_moment(wavefunction, num_spin_orbitals, num_particles)
print("Dipole moment: {:.2f} Debye".format(dipole_moment))


In [ ]:
# Plot the ground state wavefunction
plot_state(wavefunction)


In [ ]:
import numpy as np
from qiskit import Aer, execute
from qiskit.circuit.library import TwoLocal
from qiskit_nature.drivers import PySCFDriver
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.transformers import FreezeCoreTransformer, ActiveSpaceTransformer
from qiskit_nature.algorithms import GroundStateEigensolver
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter
from qiskit_nature.circuit.library import UCCSD
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors.standard_errors import depolarizing_error

# define the molecule of interest
molecule = "Be .0 .0 .0; H .0 .0 1.6; H .0 1.6 0."

# set up the PySCF driver
driver = PySCFDriver(atom=molecule)
qmolecule = driver.run()

# freeze the core and reduce the active space
freeze_core = True
remove_orbitals = [3,4]
if freeze_core:
    transformer = FreezeCoreTransformer(remove_orbitals=remove_orbitals)
else:
    transformer = ActiveSpaceTransformer(num_electrons=2, num_molecular_orbitals=6, 
                                          remove_orbitals=remove_orbitals)
problem = ElectronicStructureProblem(driver, transformer)
second_q_ops = problem.second_q_ops()
main_op = second_q_ops[0]

# convert to qubit operator
mapper = JordanWignerMapper()
converter = QubitConverter(mapper=mapper, two_qubit_reduction=True)
qubit_op = converter.convert(main_op)

# define the ansatz
ansatz = UCCSD(converter.num_qubits, converter.num_particles, converter.num_orbitals, initial_state=None)

# define the backend and execute the circuit
backend = Aer.get_backend("statevector_simulator")
quantum_instance = execute(ansatz, backend).result()

# extract the circuit and use TwoLocal as a variational form to minimize the energy
ansatz_circuit = ansatz.construct_circuit(np.zeros(ansatz.num_parameters))
optimizer = optimizer = COBYLA(maxiter=1000)
var_form = TwoLocal(rotation_blocks='ry', entanglement_blocks='cz', 
                    entanglement='full', reps=3, skip_final_rotation_layer=True)

# define the VQE algorithm
vqe = VQE(ansatz=ansatz_circuit, optimizer=optimizer, quantum_instance=backend, 
          callback=callback, initial_point=np.zeros(ansatz.num_parameters))

# define the zero-noise extrapolation method
p = [0.25, 0.5, 0.75, 1.0]
vqe_result = {}
for i in range(len(p)):
    noise_model = NoiseModel.from_backend(backend)
    for qi in noise_model.qubits:
        noise_model.add_quantum_error(depolarizing_error(0.05, 1), [qi])
    vqe_result[p[i]] = vqe.compute_minimum_eigenvalue(operator=qubit_op, aux_operators=[])

e0 = vqe_result[1.0].eigenvalue.real
zero_noise_extrapolation = (2 * vqe_result[p[0]].eigenvalue.real
                            - 5


In [22]:
import numpy as np
from qiskit import Aer
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQE
from qiskit.aqua.components.optimizers import L_BFGS_B
from qiskit.aqua.components.variational_forms import UCCSD
from qiskit.chemistry.drivers import PySCFDriver
from qiskit.chemistry.core import Hamiltonian, TransformationType, QubitMappingType
from qiskit.ignis.mitigation.measurement import (complete_meas_cal, CompleteMeasFitter)

# Define the molecule
molecule = 'Be .0 .0 .0; H .0 .0 1.6'
charge = 0
multiplicity = 1

# Use PySCF to compute the molecular Hamiltonian in the active space
driver = PySCFDriver(atom=molecule, charge=charge, spin_multiplicity=multiplicity, basis='sto3g')
q_molecule = driver.run()

# Use the Hamiltonian and a suitable qubit mapper to create a qubit operator
hamiltonian = Hamiltonian(transformation=TransformationType.FULL, qubit_mapping=QubitMappingType.PARITY,
                          two_qubit_reduction=True, freeze_core=True)
qubit_op, _ = hamiltonian.run(q_molecule)

# Choose the L-BFGS-B optimizer
optimizer = L_BFGS_B(maxfun=6000, factr=10, iprint=100)

# Choose the UCCSD ansatz for the variational form
ansatz = UCCSD(qubit_op.num_qubits, depth=1, num_orbitals=q_molecule.num_orbitals, num_particles=q_molecule.num_alpha_electrons)

# Choose the VQE algorithm to find the ground state
algorithm = VQE(qubit_op, ansatz, optimizer)

# Choose the backend to run the quantum circuit
backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend=backend)

# Run the VQE algorithm and get the result
result = algorithm.run(quantum_instance)

# Use the CompleteMeasFitter class to create the calibration matrix
cal_circuits, state_labels = complete_meas_cal(qubit_list=[i for i in range(qubit_op.num_qubits)], qr=quantum_instance.circuit.qregs[0])
cal_job = backend.run(cal_circuits, shots=8192)
meas_fitter = CompleteMeasFitter(cal_job.result(), state_labels)

# Apply the reference-state error mitigation
mitigated_result = meas_fitter.filter.apply(result['measurement'])

# Print the ground state energy and the error-mitigated energy
print(f"Ground state energy: {result['energy']:.6f} Hartree")
print(f"Error-mitigated energy: {meas_fitter.estimate/mitigated_result['total_counts']:.6f} Hartree")


ModuleNotFoundError: ignored

In [ ]:
# Extract ground state energy
print('Ground State Energy:', opt_result.fun)

# Extract optimized parameters
opt_param = opt_result.x
print('Optimized Parameters:', opt_param)


In [ ]:
import numpy as np
from qiskit import Aer
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal
from qiskit.opflow import Z2Symmetries
from qiskit.utils import QuantumInstance
from qiskit_nature.algorithms import VQEUCCFactory
from qiskit_nature.drivers import PySCFDriver
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.transformers import FreezeCoreTransformer, ActiveSpaceTransformer
from qiskit_nature.mappers.second_quantization import ParityMapper
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.algorithms import GroundStateEigensolver
from qiskit_nature.results import EigenstateResult


# Set up the PySCF driver to compute the initial state
driver = PySCFDriver(atom="Be 0 0 0; H 0 0 1.6", unit="angstrom")
molecule = driver.run()

# Freeze the core electrons and choose active orbitals
freeze_core = FreezeCoreTransformer()
frozen_molecule = freeze_core.transform(molecule)
num_particles = (frozen_molecule.num_alpha, frozen_molecule.num_beta)
num_spin_orbitals = frozen_molecule.num_orbitals
active_space = ActiveSpaceTransformer(num_electrons=num_particles, num_molecular_orbitals=8)
problem = ElectronicStructureProblem(driver, [freeze_core, active_space])

# Generate the qubit Hamiltonian
mapper = ParityMapper()
converter = QubitConverter(mapper=mapper, two_qubit_reduction=True)
qubit_op = converter.convert(problem.second_q_ops())

# Get the symmetries of the qubit Hamiltonian
z2_symmetries = Z2Symmetries.find_Z2_symmetries(qubit_op)

# Set up the VQE algorithm with UCCSD ansatz
ansatz = TwoLocal(rotation_blocks=["h", "rx", "ry"], entanglement_blocks="cz", entanglement="linear", reps=3)
optimizer = COBYLA(maxiter=1000)
vqe = VQEUCCFactory(quantum_instance=QuantumInstance(Aer.get_backend('statevector_simulator')),
                    ansatz=ansatz,
                    optimizer=optimizer,
                    include_custom=True)

# Set up the ground state eigensolver and run the algorithm
gse = GroundStateEigensolver(converter, vqe)
result = gse.solve(problem)

# Apply reference state error mitigation
ref_state = np.zeros(2 ** converter.num_qubits)
ref_state[0] = 1.0
qinstance = QuantumInstance(Aer.get_backend('qasm_simulator'), shots=8192)
ref_mitigated_result = EigenstateResult()
for res in result.raw_result.eigenstates:
    ref_mitigated_result.add_state(ref_state_mitigation(res, qinstance))

# Print the ground state energy and wavefunction
print(f"Ground state energy: {ref_mitigated_result.eigenenergies[0]}")
print(f"Ground state wavefunction: {ref_mitigated_result.eigenstates[0]}")


In [ ]:
# Define the energy operator
h2_energy = Z2 ^ Z3 + 0.5 * (X0 * X1 + Y0 * Y1) + 0.5 * (X2 * X3 + Y2 * Y3) - 2.5

# Compute the expectation value of the energy operator
energy = np.real(aer.expectation(statevector, h2_energy))
print('Corrected ground state energy:', energy)


In [ ]:
Corrected ground state energy: -14.367033046002116
